In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

In [2]:
df = pd.read_csv('WO2020-2022OCT.csv') 

In [3]:
df.head()

,wonum,siteid,status,statusdate,worktype,description,assetnum,location,changeby,changedate,...,lead,wogroup,scheduler,planner,projeng1,location2,createdby,zzcontractor,zzdepartment,passfailstatus
0,2056672,AL HASA,CLOSE,2/1/20 9:00,PM,Submersible Pump Lifting Pump -,A475-000004,KSAUHS-3-Z005-STPLA7-GFL-OSAR-EAST,SINGH,2/1/20 9:01 AM,...,H-150,2056672,H-150,H-150,H-150,NaN,MAXADMIN,C2-NESMA,WTP,NaN
1,2056954,AL HASA,CLOSE,4/27/20 12:20,PM,Uninterruptable Power Panel -,A525-001499,KSAUHS-3-Z001-COAMS-02F-ELRM-0003,NOORM,4/27/20 12:20 PM,...,H-101,2056954,H-101,H-101,H-186,NaN,MAXADMIN,C1-JASH,Electrical,NaN
2,2056955,AL HASA,CLOSE,4/27/20 12:22,PM,Power panel -,A525-001500,KSAUHS-3-Z001-COAMS-02F-ELRM-0003,NOORM,4/27/20 12:22 PM,...,H-101,2056955,H-101,H-101,H-186,NaN,MAXADMIN,C1-JASH,Electrical,NaN
3,2056961,AL HASA,CLOSE,3/30/20 15:42,PM,Television -,A530-000369,KSAUHS-3-Z002-HSBU1-01F-F117-RECP,HAFEEZS,3/30/20 3:42 PM,...,H-398,2056961,H-398,H-398,H-021,NaN,MAXADMIN,C1-JASH,Low Current,NaN
4,2057329,AL HASA,CLOSE,1/31/20 15:30,PM,Primary Pump #1 -,A435-000009,KSAUHS-3-Z004-CHPLA-GFL,ABDULRAHIM,2/1/20 10:22 AM,...,H-427,2057329,H-427,H-427,H-427,NaN,MAXADMIN,C2-NESMA,Chiller,NaN


In [4]:
df.size

29530830

In [5]:
df.shape

(843738, 35)

In [6]:
df.columns = df.columns.str.replace(' ', '')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 843738 entries, 0 to 843737
Data columns (total 35 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   wonum           843738 non-null  object 
 1   siteid          843738 non-null  object 
 2   status          843738 non-null  object 
 3   statusdate      843738 non-null  object 
 4   worktype        843486 non-null  object 
 5   description     843386 non-null  object 
 6   assetnum        753917 non-null  object 
 7   location        843596 non-null  object 
 8   changeby        843738 non-null  object 
 9   changedate      843738 non-null  object 
 10  historyflag     843738 non-null  int64  
 11  wopriority      132954 non-null  float64
 12  targcompdate    832652 non-null  object 
 13  targstartdate   832830 non-null  object 
 14  reportedby      843723 non-null  object 
 15  reportdate      843738 non-null  object 
 16  actstart        811573 non-null  object 
 17  actfinish 

In [8]:
to_remove = ['C3-SICE','LOG', 'C4-ALKHORAYEF', 'HTMS']
df = df[~(df['zzcontractor'].isin(to_remove))]

In [9]:
#delete column 

df = df.drop(['statusdate' ,'targcompdate', 'targstartdate', 'reportedby',
       'reportdate', 'actstart', 'schedstart', 'schedfinish','supervisor', 'origrecordid',
         'owner', 'lead', 'wogroup', 'scheduler', 'planner', 'projeng1', 'location2',
       'createdby', 'passfailstatus', "persongroup" , "ownergroup", 'changeby', 'changedate', 'description','zzcontractor'] , axis=1)

In [10]:
df.shape

(820109, 10)

In [11]:
df.columns

Index(['wonum', 'siteid', 'status', 'worktype', 'assetnum', 'location',
       'historyflag', 'wopriority', 'actfinish', 'zzdepartment'],
      dtype='object')

In [12]:
df = df.rename(columns={"wonum": "work_number", 
                   "assetnum": "asset_ID",
                   "wopriority": "priority",
                   "actfinish": "date", 
                   "historyflag": "modifications",
                   'zzdepartment': "field_type",
                   "zzcontractor": "contractor"}) 

In [13]:
df.columns

Index(['work_number', 'siteid', 'status', 'worktype', 'asset_ID', 'location',
       'modifications', 'priority', 'date', 'field_type'],
      dtype='object')

In [14]:
df['worktype'].value_counts()

PM     688294
CM      95936
PDI     33685
EM       1773
MOD       166
LOG         1
HSE         1
CMP         1
Name: worktype, dtype: int64

In [15]:
to_remove = ['PM','LOG', '2020', 'CMP', '2021', 'HSE']
df = df[~(df['worktype'].isin(to_remove))]

In [16]:
df.shape

(131812, 10)

In [17]:
df['worktype'].value_counts()

CM     95936
PDI    33685
EM      1773
MOD      166
Name: worktype, dtype: int64

In [18]:
df['status'].value_counts()

CLOSE           125665
CAN               2439
ACCESSDENIED      1323
COMP              1113
WMATL              559
WAPPR              153
WPLAN              143
SCHEDCOMP           95
REQCAN              76
WASSESS             61
WSCH                53
CANREJ              44
INPRG               40
PLANCOMPWMAT        13
WSAFE               13
DEFERRED             7
READY                5
PLANCOMP             5
TAAPPR               2
FAIL                 2
PASS                 1
Name: status, dtype: int64

In [19]:
df.drop(df[ (df['status'] != 'CLOSE') & (df['status'] != 'COMP') ].index , inplace=True)

In [20]:
df['status'].value_counts()

CLOSE    125665
COMP       1113
Name: status, dtype: int64

In [21]:
df['date']

5         5/19/20 8:08 PM
6          1/1/20 3:52 PM
68        1/12/20 1:28 PM
69         1/8/20 2:46 PM
70         1/9/20 7:21 AM
               ...       
843499     3/3/20 1:57 PM
843504    2/27/20 1:24 PM
843508    2/25/20 7:09 PM
843514    3/3/20 11:06 PM
843524     3/3/20 9:51 PM
Name: date, Length: 126778, dtype: object

In [22]:
df.shape

(126778, 10)

In [23]:
df.size

1267780

In [25]:
df['asset_ID'].isnull().sum()

83193

In [26]:
df['asset_ID']

5         A530-003865
6         A530-007949
68        A530-006123
69                NaN
70                NaN
             ...     
843499            NaN
843504            NaN
843508            NaN
843514            NaN
843524            NaN
Name: asset_ID, Length: 126778, dtype: object

In [27]:
df['asset_ID'] = df.asset_ID.interpolate(method='pad')

In [28]:
df['asset_ID'].isnull().sum()

0

In [29]:
df.head()

,work_number,siteid,status,worktype,asset_ID,location,modifications,priority,date,field_type
5,2057394,AL HASA,CLOSE,CM,A530-003865,KSAUHS-3-Z004-STD-GFL-IDFR-0001,1,3.0,5/19/20 8:08 PM,Low Current
6,2057395,AL HASA,CLOSE,CM,A530-007949,KSAUHS-3-Z001-COAMS-02F,1,3.0,1/1/20 3:52 PM,Low Current
68,2076165,AL HASA,CLOSE,CM,A530-006123,KSAUHS-3-Z002-HSBU2-GFL-F114-RECP,1,3.0,1/12/20 1:28 PM,Low Current
69,2076166,AL HASA,CLOSE,CM,A530-006123,KSAUHS-3-Z001-ADMIN-GFL,1,3.0,1/8/20 2:46 PM,HVAC
70,2076203,AL HASA,CLOSE,CM,A530-006123,KSAUHS-3-Z003-V5/13A-GFL-KTCN-0001,1,2.0,1/9/20 7:21 AM,LPG


In [30]:
df.isnull().sum()

work_number         0
siteid              0
status              0
worktype           39
asset_ID            0
location           27
modifications       0
priority         2190
date                0
field_type         91
dtype: int64

In [31]:
df.describe()

,modifications,priority
count,126778.000000,124588.000000
mean,0.991221,2.946873
std,0.093285,0.283018
min,0.000000,1.000000
25%,1.000000,3.000000
50%,1.000000,3.000000
75%,1.000000,3.000000
max,1.000000,3.000000


In [32]:
df = df.dropna()

In [33]:
df.isnull().sum()

work_number      0
siteid           0
status           0
worktype         0
asset_ID         0
location         0
modifications    0
priority         0
date             0
field_type       0
dtype: int64

In [34]:
df.shape

(124563, 10)

In [35]:
#Deletes the time and keeps only the date
df['date'] = pd.to_datetime(df['date']).dt.date

In [36]:
#adding year and month columns
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month

In [37]:
df.head()

,work_number,siteid,status,worktype,asset_ID,location,modifications,priority,date,field_type,year,month
5,2057394,AL HASA,CLOSE,CM,A530-003865,KSAUHS-3-Z004-STD-GFL-IDFR-0001,1,3.0,2020-05-19,Low Current,2020,5
6,2057395,AL HASA,CLOSE,CM,A530-007949,KSAUHS-3-Z001-COAMS-02F,1,3.0,2020-01-01,Low Current,2020,1
68,2076165,AL HASA,CLOSE,CM,A530-006123,KSAUHS-3-Z002-HSBU2-GFL-F114-RECP,1,3.0,2020-01-12,Low Current,2020,1
69,2076166,AL HASA,CLOSE,CM,A530-006123,KSAUHS-3-Z001-ADMIN-GFL,1,3.0,2020-01-08,HVAC,2020,1
70,2076203,AL HASA,CLOSE,CM,A530-006123,KSAUHS-3-Z003-V5/13A-GFL-KTCN-0001,1,2.0,2020-01-09,LPG,2020,1


In [38]:
df['field_type'].unique()

array(['Low Current', 'HVAC', 'LPG', 'Electrical', 'Civil', 'Plumbing',
       'BMS', 'STP', 'Elevator', 'WTP', 'PMG', 'Fire Alarm', 'Chiller',
       'Non-Medical', 'Generator', 'Safety', 'Appliance', 'Electronic',
       'STP Operations', 'Mitsubishi', 'OCL & Depot', 'CSSD Non-Medical',
       'Mechanical'], dtype=object)

In [39]:
df['field_type'].value_counts()

Electrical          41564
Civil               21556
HVAC                21145
Plumbing            19308
BMS                  5778
LPG                  4009
Electronic           3111
Low Current          2117
Appliance            1278
Fire Alarm           1095
PMG                   797
WTP                   740
Non-Medical           580
Elevator              551
Safety                253
Mitsubishi            238
Chiller               220
STP Operations         85
CSSD Non-Medical       58
Generator              50
STP                    27
Mechanical              2
OCL & Depot             1
Name: field_type, dtype: int64

In [40]:
#Replacing

ELECTRICAL = ['Low Current','Generator','Appliance', 'BMS', 'Electronic'] #Electrical
MECHANICAL = ['HVAC', 'LPG', 'STP', 'Elevator', 'WTP', 'PMG', 'Fire Alarm', 'Chiller',
              'STP Operations', 'Mitsubishi', 'OCL & Depot', 'Safety'] #Mechanical

df['field_type'].replace(
    to_replace=ELECTRICAL,
    value='Electrical',
    inplace=True
)

df['field_type'].replace(
    to_replace=MECHANICAL,
    value='Mechanical',
    inplace=True
)

df['field_type'] = df.field_type.str.replace('CSSD ', '')

df['field_type'].value_counts()

Electrical     53898
Mechanical     29163
Civil          21556
Plumbing       19308
Non-Medical      638
Name: field_type, dtype: int64

In [41]:
df['location' ] = df['location' ].str[:13]

In [42]:
df['location' ]

5         KSAUHS-3-Z004
6         KSAUHS-3-Z001
68        KSAUHS-3-Z002
69        KSAUHS-3-Z001
70        KSAUHS-3-Z003
              ...      
843499    KSAUHS-1-Z01A
843504    KSAUHS-1-Z02A
843508    KSAUHS-1-Z04A
843514    KSAUHS-1-Z01B
843524    KSAUHS-1-Z04A
Name: location, Length: 124563, dtype: object